In [70]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import folium
import branca.colormap as cm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
pd.set_option('display.max_columns', None)

In [87]:
url = 'https://www.immoscout24.ch/de/wohnung/mieten/ort-zuerich?pn=2&r=10'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
raw_data = soup.find(id='state')

jason_data = json.loads(raw_data.string[18:])
#raw_data.string[18:1000]

JSONDecodeError: Expecting value: line 1 column 4160 (char 4159)

In [75]:
# getting the published empty flats of Zurich in a 10km radius
result_list = []
max = 20

for i in range(max):
    url = 'https://www.immoscout24.ch/de/wohnung/mieten/ort-zuerich?pn={}&r=10'.format(i+1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    raw_data = soup.find(id='state')
    jason_data = json.loads(raw_data.string[18:])
    data_list = jason_data['pages']['searchResult']['resultData']['listData']

    page_list = []

    for elem in data_list:
        if 'price' in elem: price = elem['price']
        else: price = np.nan

        if 'surfaceLiving' in elem: sl = elem['surfaceLiving']
        else: sl = np.nan

        if 'numberOfRooms' in elem: nor = elem['numberOfRooms']
        else: nor = np.nan

        if 'title' in elem: ttl = elem['title']
        else: ttl = np.nan

        if 'latitude' in elem: lat = elem['latitude']
        else: lat = np.nan

        if 'longitude' in elem: lon = elem['longitude']
        else: lon = np.nan

        page_list.append([elem['id'], price, nor, sl, ttl, elem['zip'], lat, lon])

    result_list += page_list

result_list[10:20]

JSONDecodeError: Expecting value: line 1 column 4697 (char 4696)

In [27]:
flats_df = pd.DataFrame(result_list)
flats_df.columns = ['ID', 'Price', 'RoomNr', 'Surface', 'Title', 'ZIP', 'Latitude', 'Longitude']

#flats_df.dropna(inplace=True)
flats_df['Title'] = flats_df['Title'].str.lower()
flats_df['Title'].str.contains("blick|sicht|anstoss").value_counts()

flats_df['Title'].str.contains("charm").value_counts()

flats_df['Vista'] = 0
filter1 = flats_df['Title'].str.contains('blick|sicht|anstoss')
filter2 = ~flats_df['Title'].str.contains('besichtigung')
flats_df.loc[filter1 & filter2, 'Vista'] = 1

flats_df['Bright'] = 0
filter = flats_df['Title'].str.contains("hell|licht|sonnig")
flats_df.loc[filter, 'Bright'] = 1

flats_df

ValueError: Length mismatch: Expected axis has 0 elements, new values have 8 elements

In [28]:
X_df = flats_df.set_index('ID')
X_df = X_df[['Price', 'RoomNr', 'Surface', 'Vista', 'Bright']]
X_df.dropna(inplace=True)
#X = normalize(X_df.to_numpy())
X = X_df.to_numpy()
clusters = KMeans(n_clusters=4, random_state=170).fit_predict(X)
X_df['Cat'] = clusters


KeyError: "None of ['ID'] are in the columns"

In [29]:
X_df[X_df['Cat']==0].describe()

NameError: name 'X_df' is not defined

In [30]:
# create map of zurich using latitude and longitude values
map_zurich = folium.Map(location=[47.38, 8.53], zoom_start=11.5)

linear = cm.LinearColormap(
    ['green', 'red'],
    vmin=800, vmax=6000
)

linear

# add markers to map
for lat, lng, price in zip(flats_df['Latitude'], flats_df['Longitude'], flats_df['Price']):
    label = folium.Popup(str(price), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=linear(price),
        fill=True,
        fill_color=linear(price),
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)

map_zurich

KeyError: 'Latitude'

In [31]:
flats_sample = flats_df.sample(100, random_state=5)
#flats_sample.set_index('ID', inplace=True)
flats_sample

ValueError: a must be greater than 0 unless no samples are taken

In [32]:
CLIENT_ID = 'VV5H111QRCSXOLD5LF52CDDBI1S1QPDX5BFBOI3IJQHGYI4L' # your Foursquare ID
CLIENT_SECRET = 'TEH4SQMZBUB1VHCJZQYNLNHCQTBLTEXIHBLY1VWM1FMSYU1N' # your Foursquare Secret
VERSION = '20180604'
RADIUS = 500
LIMIT = 50

In [33]:
venues_list=[]

for flatID, flatLat, flatLon in zip(flats_sample['ID'], flats_sample['Latitude'], flats_sample['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, flatLat, flatLon, VERSION, RADIUS, LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    for v in results:
        venues_list.append([
            flatID,
            v['venue']['categories'][0]['name']
        ])

venues_df = pd.DataFrame(venues_list)
venues_df.columns = ['ID', 'Category']

onehot = pd.get_dummies(venues_df['Category'],  prefix="", prefix_sep="")
onehot['ID'] = venues_df['ID']
onehot = onehot.groupby('ID').sum()
onehot

NameError: name 'flats_sample' is not defined

In [34]:
onehot.reset_index(inplace=True)
onehot = onehot.groupby('ID').sum()
#onehot.drop('index', inplace=True)
onehot

NameError: name 'onehot' is not defined

In [35]:
pois = [
    'Bus Station',
    'Bus Stop',
    'Convenience Store',
    'Discount Store',
    'Grocery Store',
    'Light Rail Station',
    #'Market',
    'Shopping Mall',
    'Supermarket',
    'Train Station'
]

onehot = onehot[pois]
onehot['sum'] = onehot.sum(axis=1)
onehot

NameError: name 'onehot' is not defined

In [36]:
#flats_sample.set_index('ID', inplace=True)
flats_joined = flats_sample.join(onehot['sum'], how='left')
flats_joined['sum'].value_counts()

filter = flats_joined['RoomNr'] == 3.5
flats_filtered = flats_joined[filter]


plt.scatter(flats_filtered['sum'], flats_filtered['Price'])
flats_filtered.corr()

NameError: name 'flats_sample' is not defined